In [ ]:
# Program to determine costs from actuals (FDS) and orders (FourthShift)

import pandas as pd
import numpy as np

localpath = r"C:\Users\kashir\Documents\SnOP\00_Model"
local_order_file = r'\FS_Orders.xls'
fs_orders_file = localpath + local_order_file

# Lookup file
lookups_file = localpath + '\SnOP Lookups.xlsx'
lookups_xls = pd.ExcelFile(lookups_file)

#... actuals dataframe
#_____________________

actuals_file = localpath + '\Actuals.xlsx'
df_actuals = pd.read_excel(actuals_file, 'Actuals', skiprows=4, header=0)
df_actuals.By = pd.to_datetime(df_actuals.By).dt.date

df_actuals.DataType = df_actuals.assign(DataType='Actual') # Change from EstSales

#... filter products only in the TLAs
df_prodref = pd.read_excel(lookups_xls, 'ProductRef', header=0, skiprows=4)

#...get the ams and integrate with actuals
df_am = pd.read_excel(lookups_xls, 'AM_names', header=0).drop_duplicates('AM Code').set_index('AM Code')

In [146]:
df_cost = df_actuals.set_index('AM')
df_cost = df_cost.merge(df_am, left_index=True, right_index=True)
df_cost = df_cost.reset_index(drop=True).rename(columns={'AM Name': 'AM'})

In [148]:
#...get unit costs
df_cost = df_cost.assign(TLA=df_cost.ProductID.str.slice(0,3))

# keep only TLAs from the product reference
df_cost1 = df_cost[df_cost.TLA.isin(df_prodref.TLA)]

# keep only positive quantities
df_cost1 = df_cost1[df_cost1.Qty > 0]

# remove unnecessary rows from ProductID
x_prod = ['SAM', 'CAPSULE', 'INNER']
df_cost1 = df_cost1[~df_cost1.ProductID.str.contains('|'.join(x_prod))]

# remove Qty = 0
df_cost1 = df_cost1[df_cost1.Qty != 0.0]

df_cost1 = df_cost1.assign(uGBPValK= df_cost1.GBPValue/df_cost1.Qty*1000)
df_cost1 = df_cost1.assign(uGBPCostK= df_cost1.StdGBPCost/df_cost1.Qty*1000)
df_cost1 = df_cost1.assign(uMaterialK= df_cost1.Material/df_cost1.Qty*1000)
df_cost1 = df_cost1.assign(uLabourK= df_cost1.Labour/df_cost1.Qty*1000)
df_cost1 = df_cost1.assign(uVarohK= df_cost1.VarOH/df_cost1.Qty*1000)
df_cost1 = df_cost1.assign(uFixedohK= df_cost1.FixedOH/df_cost1.Qty*1000)

# Compute the margin
df_cost1 = df_cost1.assign(uMargin=(df_cost1.GBPValue - df_cost1.StdGBPCost)/df_cost1.GBPValue)

# Remove margin = inf (i.e. StdGBPCost = 0) data
df_cost1 = df_cost1.replace([np.inf, -np.inf], np.nan).dropna(subset=['uMargin'], how='all').reset_index(drop=True)

df_cost1.to_csv(path_or_buf=localpath+'\_costing.csv', index=None)

In [ ]:
#... get the most recent costs
df_last_tla = df_cost1.set_index('By', 'Site', 'TLA').sort_index().drop_duplicates(['Site', 'TLA'], keep='last').reset_index(drop=True)
df_last_product = df_cost1.set_index('By', 'Site', 'ProductID').sort_index().drop_duplicates(['Site', 'ProductID'], keep='last').reset_index(drop=True)

In [ ]:
df_last_tla.set_index(['TLA', 'Site']).sort_index()